<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting_v1/blob/main/Value_Buy_Sell_Best_Prospects.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance
!pip install pandas-ta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas-ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218910 sha256=08a508ef813a6f143f6b1786045819bd47c7761fa0e6a704afdeffa71a2c0d8a
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas-ta


In [20]:
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_ta as ta

def find_buy_sell_levels(stock_list):
    results = []

    for stock in stock_list:
        # Download historical data from yfinance
        data = yf.download(stock + '.NS', period='5y', interval='1d', progress=False)

        if data.empty:
            print(f"No data found for {stock}")
            continue

        # Calculate 200-day SMA and 14-day RSI
        data['Close'] = round(data['Close'],2)
        data['SMA_200'] = round(data['Close'].rolling(window = 200, min_periods = 1).mean(),0)
        data.ta.rsi(append=True)
        data['RSI_14'] = round(data['RSI_14'],0)

        # All-time high (ATH)
        data['ATH'] = data['Close'].cummax()

        buy_triggered = False
        buy_date = None
        for i in range(len(data)):
            if not buy_triggered:
                # Buy criteria: Price < 200 SMA and RSI < 30 for the first time
                if data['Close'].iloc[i] < data['SMA_200'].iloc[i] and data['RSI_14'].iloc[i] < 30:
                    buy_triggered = True
                    buy_date = data.index[i]
                    results.append({
                        'Stock': stock,
                        'Date': buy_date,
                        'Close': data['Close'].iloc[i],
                        '200 SMA': data['SMA_200'].iloc[i],
                        'RSI 14': data['RSI_14'].iloc[i],
                        'Action': 'Buy'
                    })
            else:
                # Sell criteria: Price reaches ATH after buying
                if data['Close'].iloc[i] >= data['ATH'].iloc[i]:
                    sell_date = data.index[i]
                    results.append({
                        'Stock': stock,
                        'Date': sell_date,
                        'Close': data['Close'].iloc[i],
                        '200 SMA': data['SMA_200'].iloc[i],
                        'RSI 14': data['RSI_14'].iloc[i],
                        'Action': 'Sell'
                    })
                    buy_triggered = False  # Reset for next buy/sell cycle

    # Return the results as a DataFrame
    return pd.DataFrame(results)

def get_combined_signals(result):
    # Filter buy and sell actions
    buy_signals = result[result['Action'] == 'Buy'].copy()
    sell_signals = result[result['Action'] == 'Sell'].copy()

    # Sort both buy and sell signals by stock and date
    buy_signals = buy_signals.sort_values(by=['Stock', 'Date']).reset_index(drop=True)
    sell_signals = sell_signals.sort_values(by=['Stock', 'Date']).reset_index(drop=True)

    # Create an empty DataFrame to hold the combined signals
    combined_signals = []

    # Iterate over each stock to ensure correct pairing of Buy and Sell signals
    stocks = buy_signals['Stock'].unique()

    for stock in stocks:
        # Get buy and sell signals for the stock
        buy_stock = buy_signals[buy_signals['Stock'] == stock].reset_index(drop=True)
        sell_stock = sell_signals[sell_signals['Stock'] == stock].reset_index(drop=True)

        # Pair each buy with the next available sell after the buy date
        i, j = 0, 0
        while i < len(buy_stock):
            if j < len(sell_stock) and sell_stock.loc[j, 'Date'] > buy_stock.loc[i, 'Date']:
                # Valid pair of buy and sell
                combined_signals.append({
                    'Stock': stock,
                    'Buy Date': buy_stock.loc[i, 'Date'],
                    'Buy Price': buy_stock.loc[i, 'Close'],
                    'Sell Date': sell_stock.loc[j, 'Date'],
                    'Sell Price': sell_stock.loc[j, 'Close'],
                    'Gain %': round(((sell_stock.loc[j, 'Close'] - buy_stock.loc[i, 'Close']) / buy_stock.loc[i, 'Close']) * 100,2),
                    'Days Held': (sell_stock.loc[j, 'Date'] - buy_stock.loc[i, 'Date']).days
                })
                # Move to the next buy and sell pair
                i += 1
                j += 1
            else:
                # If no valid sell is found, append buy action and keep sell-related fields as None
                combined_signals.append({
                    'Stock': stock,
                    'Buy Date': buy_stock.loc[i, 'Date'],
                    'Buy Price': buy_stock.loc[i, 'Close'],
                    'Sell Date': None,
                    'Sell Price': None,
                    'Gain %': None,
                    'Days Held': None
                })
                # Move to the next buy action
                i += 1

    # Convert the combined signals list to a DataFrame
    combined_signals_df = pd.DataFrame(combined_signals)

    # Return the final result
    return combined_signals_df

# Example usage
if __name__ == "__main__":
    mypf = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')

    mypf = mypf[(mypf['Criteria'] == 'X40') | (mypf['Criteria'] == 'X40N')]

    stock_n100 = mypf['Symbol'].unique()

    exclude = ['CADILAHC','MMTC','JIOFIN','SCHAEFFLER ','ADANITRANS', 'EQUITAS', 'MCDOWELL-N', 'ARE&M', 'UJJIVAN']

    stock_n100 = mypf[~mypf['Symbol'].isin(exclude) ]['Symbol'].unique()

    stock_ids = stock_n100
    result = find_buy_sell_levels(stock_ids)
    combined_result = get_combined_signals(result)

combined_result.sort_values(by = 'Buy Date', ascending=False).head(30)

,Stock,Buy Date,Buy Price,Sell Date,Sell Price,Gain %,Days Held
153,SANOFI,2024-06-13,6513.05,NaT,NaN,NaN,NaN
13,ANGELONE,2024-05-09,2420.35,NaT,NaN,NaN,NaN
170,TITAN,2024-05-06,3280.15,NaT,NaN,NaN,NaN
71,HCLTECH,2024-04-30,1366.60,2024-08-20,1686.75,23.43,112.0
152,SANOFI,2024-03-21,7535.75,2024-06-07,9573.65,27.04,78.0
5,5PAISA,2024-03-13,472.45,NaT,NaN,NaN,NaN
54,EQUITASBNK,2024-03-12,92.95,NaT,NaN,NaN,NaN
174,UJJIVANSFB,2024-03-12,49.20,NaT,NaN,NaN,NaN
10,AKZOINDIA,2024-03-05,2452.80,2024-06-24,2932.60,19.56,111.0
134,PGHH,2024-02-28,15752.50,NaT,NaN,NaN,NaN


In [21]:
combined_result.sort_values(by = 'Sell Date', ascending=False).head(30)

,Stock,Buy Date,Buy Price,Sell Date,Sell Price,Gain %,Days Held
131,PFIZER,2021-10-20,5261.70,2024-09-02,6318.30,20.08,1048.0
95,INFY,2022-04-18,1621.40,2024-08-30,1943.70,19.88,865.0
47,DABUR,2021-12-20,558.20,2024-08-26,654.55,17.26,980.0
80,HINDUNILVR,2021-10-27,2396.60,2024-08-23,2815.60,17.48,1031.0
63,GILLETTE,2019-11-18,7014.80,2024-08-21,8266.60,17.85,1738.0
71,HCLTECH,2024-04-30,1366.60,2024-08-20,1686.75,23.43,112.0
43,CERA,2024-01-03,7776.95,2024-08-06,9833.60,26.45,216.0
160,SYMPHONY,2021-07-28,951.75,2024-08-06,1462.15,53.63,1105.0
92,ICICIPRULI,2021-12-20,541.45,2024-07-26,724.90,33.88,949.0
100,ITC,2024-02-08,414.55,2024-07-23,492.20,18.73,166.0
